In [28]:
import struct
import numpy as np
import math
import matplotlib.pyplot as plt
import random
import os, struct
import sys
import time
from sklearn.neighbors import NearestNeighbors
from collections import deque
from array import array as pyarray
from numpy import append, array, int8, uint8, zeros

In [3]:
trainingImagePath='dataset/train-images.idx3-ubyte'
trainingLablePath='dataset/train-labels.idx1-ubyte'
testImagePath='dataset/t10k-images.idx3-ubyte'
testLablePath='dataset/t10k-labels.idx1-ubyte'

In [38]:
class PrototypeSelectionForNearestNeighbor():
    def __init__(self):
        self.trainingImages = None
        self.trainingLabels = None
        self.testImages = None
        self.testLabels = None
        self.randomImages = []
        self.randomLabels = []
        
    def load_mnist(self, dataset="training", digits=np.arange(10), path="dataset/"):
        """
        Loads MNIST files into 3D numpy arrays

        Adapted from: http://abel.ee.ucla.edu/cvxopt/_downloads/mnist.py
        """

        if dataset == "training":
            fname_img = os.path.join(path, 'train-images-idx3-ubyte')
            fname_lbl = os.path.join(path, 'train-labels-idx1-ubyte')
        elif dataset == "testing":
            fname_img = os.path.join(path, 't10k-images-idx3-ubyte')
            fname_lbl = os.path.join(path, 't10k-labels-idx1-ubyte')
        else:
            raise ValueError("dataset must be 'testing' or 'training'")

        flbl = open(fname_lbl, 'rb')
        magic_nr, size = struct.unpack(">II", flbl.read(8))
        lbl = pyarray("b", flbl.read())
        flbl.close()

        fimg = open(fname_img, 'rb')
        magic_nr, size, rows, cols = struct.unpack(">IIII", fimg.read(16))
        img = pyarray("B", fimg.read())
        fimg.close()

        ind = [ k for k in range(size) if lbl[k] in digits ]
        N = len(ind)

        images = zeros((N, rows, cols), dtype=int)
        labels = zeros((N, 1), dtype=int)
        for i in range(len(ind)):
            images[i] = array(img[ ind[i]*rows*cols : (ind[i]+1)*rows*cols]).reshape((rows, cols))
            labels[i] = lbl[ind[i]]
        images = map(lambda x: np.concatenate(x),list(images))
        labels = map(lambda x:x[0],list(labels))
        return images, labels
    
    def showImage(self, imageArray):
        fig = plt.figure()
        plotwindow = fig.add_subplot(111)
        plt.imshow(imageArray, cmap='gray')
        plt.show()
    
    def loadDataset(self):
        self.trainingImages, self.trainingLabels = self.load_mnist(dataset = "training")
        self.testImages, self.testLabels = self.load_mnist(dataset = "testing")
    
    def findRandomSubset(self, numSubset, trainingImages, trainingLabels):
        randomImages, randomLabels = [],[]
        for i in range(numSubset):
            x = random.randint(0,len(trainingImages)-1)
            randomImages.append(trainingImages[x])
            randomLabels.append(trainingLabels[x])
        return randomImages, randomLabels

    def removeOutlier(self, trainingImages, trainingLabels):
        nbrs = NearestNeighbors(n_neighbors=2, algorithm='auto').fit(trainingImages)
        indices = nbrs.kneighbors(trainingImages, return_distance = False)
        notMatch = 0
        for indexPair in list(indices):
            rawTrainingLabels = trainingLabels[:]
            if rawTrainingLabels[indexPair[0]] != rawTrainingLabels[indexPair[1]]:
                notMatch += 1
                print notMatch,
        print 'Outliers:', notMatch
        needPopIndexPair = filter(lambda indexPair: trainingLabels[indexPair[0]] != trainingLabels[indexPair[1]],list(indices))
        needPopIndex = map(lambda pair: pair[0],needPopIndexPair)
        needPopIndex.sort(reverse = True)
        for index in needPopIndex:
            trainingImages.pop(index)
            trainingLabels.pop(index)
        print len(trainingImages),len(trainingLabels)
        return trainingImages, trainingLabels
    
    def createSubset(self, numSubset, trainingImages, trainingLabels):
        i = random.randint(0,len(trainingImages)-1)
        subsetImages = [trainingImages[i]]
        subsetLabels = [trainingLabels[i]]

        for numberIter in range(3*len(trainingImages)):
            i = random.randint(0,len(trainingImages)-1)
            nearestIdx = findNearestNeighbor(trainingImages[i], subsetImages)
            # if nearest prototype from subset has a different label than n th training img.
            # Add it to subset
            if trainingLabels[i] != subsetLabels[nearestIdx]:
                subsetImages.append(trainingImages[i]) #Add it to subset
                subsetLabels.append(trainingLabels[i])
                trainingImages.pop(i)
                trainingLabels.pop(i)
                numSubset -= 1
                if numSubset%500 == 0:
                    print "Need select ",numSubset," subset element"
            if numSubset == 0:
                break
        if numSubset != 0:
            for i in range(numSubset-1):
                x = random.randint(0,len(trainingImages)-1)
                subsetImages.append(trainingImages[x])
                subsetLabels.append(trainingLabels[x])

        return subsetImages, subsetLabels

    def findNearestNeighbor(self, testImage, trainingImages):
        nearestIdx = -1
        nearestDistance = sys.maxint
        for j in range(len(trainingImages)):
            distance = np.linalg.norm(testImage - trainingImages[j])
            if distance < nearestDistance:
                nearestDistance = distance
                nearestIdx = j
        return nearestIdx
    
    def testNearestNeighbor(self, trainingImages, trainingLabels, testImages, testLabels):
        nbrs = NearestNeighbors(n_neighbors=1, algorithm='auto').fit(trainingImages)
        indices = nbrs.kneighbors(testImages,return_distance=False)
        correctness = 0
        for i in range(0,len(testLabels)):
            if testLabels[i] == trainingLabels[indices[i][0]]:
                correctness += 1
        return correctness*1.0/len(testLabels)

In [36]:
prototypeSelection = PrototypeSelectionForNearestNeighbor()
trainingImages, trainingLabels = prototypeSelection.load_mnist(dataset = 'training')
testImages, testLabels = prototypeSelection.load_mnist(dataset = 'testing')
randomImages, randomLabels = prototypeSelection.findRandomSubset(1000, trainingImages, trainingLabels)
prototypeSelection.testNearestNeighbor(randomImages, randomLabels, testImages, testLabels)

0.8866

In [ ]:
filteredTrainingImages, filteredTrainingLabels = prototypeSelection.removeOutlier(trainingImages, trainingLabels)

In [12]:
from collections import deque
def removeOutlier(trainingImages, trainingLabels):
    qTrainingImages = deque(trainingImages)
    qTrainingLabels = deque(trainingLabels)
    notMatch = 0

    for loop in range(len(trainingImages)):
        dataImage = qTrainingImages.popleft()
        dataLabel = qTrainingLabels.popleft()
        nearestIdx = findNearestNeighbor(dataImage, qTrainingImages)
        if qTrainingLabels[nearestIdx] == dataLabel:
            qTrainingImages.append(dataImage)
            qTrainingLabels.append(dataLabel)
        else:
            notMatch += 1
        if loop % 50 == 0:
            print loop,notMatch,"\\",
    return qTrainingImages, qTrainingLabels                

In [10]:
nbrs = NearestNeighbors(n_neighbors=2, algorithm='auto').fit(trainingImages)
distances, indices = nbrs.kneighbors(trainingImages)

notMatch = 0
for indexPair in list(indices):
    rawTrainingLabels = trainingLabels[:]
    if rawTrainingLabels[indexPair[0]] != rawTrainingLabels[indexPair[1]]:
        notMatch += 1
print 'notmatch',notMatch

In [83]:
needPopIndexPair = filter(lambda indexPair: trainingLabels[indexPair[0]] != trainingLabels[indexPair[1]],list(indices))
needPopIndex = map(lambda pair: pair[0],needPopIndexPair)
needPopIndex.sort(reverse = True)
for index in needPopIndex:
    trainingImages.pop(index)
    trainingLabels.pop(index)
print len(trainingImages),len(trainingLabels)

58423 58423


In [84]:
def createSubset(numSubset,trainingImages, trainingLabels):
    i = random.randint(0,len(trainingImages)-1)
    subsetImages = [trainingImages[i]]
    subsetLabels = [trainingLabels[i]]

    for numberIter in range(3*len(trainingImages)):
        i = random.randint(0,len(trainingImages)-1)
        nearestIdx = findNearestNeighbor(trainingImages[i], subsetImages)
        # if nearest prototype from subset has a different label than n th training img.
        # Add it to subset
        if trainingLabels[i] != subsetLabels[nearestIdx]:
            subsetImages.append(trainingImages[i]) #Add it to subset
            subsetLabels.append(trainingLabels[i])
            trainingImages.pop(i)
            trainingLabels.pop(i)
            numSubset -= 1
            if numSubset%500 == 0:
                print "Need select ",numSubset," subset element"
        if numSubset == 0:
            break
    if numSubset != 0:
        for i in range(numSubset-1):
            x = random.randint(0,len(trainingImages)-1)
            subsetImages.append(trainingImages[x])
            subsetLabels.append(trainingLabels[x])

    return subsetImages, subsetLabels

In [65]:
subsetImages,subsetLabels = createSubset(1000, trainingImages, trainingLabels)

Need select  500  subset element
Need select  0  subset element


In [66]:
testNN(testImages,testLabels,subsetImages,subsetLabels)

0.9031

In [85]:
subsetImages,subsetLabels = createSubset(5000, trainingImages[:], trainingLabels[:])

Need select  4500  subset element
Need select  4000  subset element
Need select  3500  subset element
Need select  3000  subset element
Need select  2500  subset element
Need select  2000  subset element
Need select  1500  subset element
Need select  1000  subset element


In [89]:
print len(subsetImages)

5000


In [92]:
testNN(testImages,testLabels,subsetImages,subsetLabels)

finished 1000 testcase 0.936
finished 2000 testcase 0.9315
finished 3000 testcase 0.929333333333
finished 4000 testcase 0.9295
finished 5000 testcase 0.9278
finished 6000 testcase 0.9335
finished 7000 testcase 0.937857142857
finished 8000 testcase 0.942375
finished 9000 testcase 0.945666666667


0.9464

In [94]:
subsetImages,subsetLabels = createSubset(10000, trainingImages[:], trainingLabels[:])

Need select  9500  subset element
Need select  9000  subset element
Need select  8500  subset element
Need select  8000  subset element
Need select  7500  subset element
Need select  7000  subset element
Need select  6500  subset element
Need select  6000  subset element


In [102]:
testNN(testImages,testLabels,subsetImages,subsetLabels)

IndexError: list index out of range

In [ ]:
nbrs = NearestNeighbors(n_neighbors=2, algorithm='auto').fit(subsetImages)
nearestIdx = nbrs.kneighbors(testImages)

In [106]:
print list(nearestIdx)[0]

[[  793.98677571]
 [ 1330.79637811]
 [  416.66533333]
 ..., 
 [ 1224.39331916]
 [ 1406.74340233]
 [ 1487.49521008]]


In [105]:
match = 0
for i in range(len(list(nearestIdx))):
   if testLabels[i] == subsetLabels[nearestIdx[i][0]]:
    match += 1
print match*1.0/10000

TypeError: only integer arrays with one element can be converted to an index

In [104]:
notMatchIndexPair = filter(lambda indexPair: testLabels[indexPair[0]] != trainingLabels[indexPair[1]],list(nearestIdx))
numNotMatch = len(notMatchIndexPair)
print numNotMatch*1.0/10000

TypeError: only integer arrays with one element can be converted to an index